# Módulo 2: Introducción a Llama 3.1

## Demostración práctica de las capacidades y características de Llama 3.1

**Autor:** Pablo Álvarez 
**Fecha:** Julio 2025

---

### 🚀 **Compatible con Kaggle**

Este notebook está optimizado para ejecutarse en:
- **Kaggle Notebooks** (recomendado para GPU gratuita)
- **Google Colab**
- **Entorno local**

### 🔑 **Configuración de Token HF (Variables de Ambiente):**

Este notebook lee el token de Hugging Face directamente desde las **variables de ambiente del sistema operativo**:

**Para Kaggle:**
1. Ve a **Settings > Secrets** en tu notebook
2. Agrega `HF_TOKEN` con tu token de Hugging Face
3. El notebook lo detectará automáticamente

**Para Google Colab:**
1. Ejecuta: `import os`
2. Ejecuta: `os.environ['HF_TOKEN'] = 'tu_token_aqui'`

**Para entorno local:**
1. `export HF_TOKEN=tu_token_aqui`
2. O agrega al archivo `.bashrc`/`.zshrc`

**Obtener token:** https://huggingface.co/settings/tokens

### 🚀 **Activar GPU (Kaggle):**
- Ve a **Settings > Accelerator**
- Selecciona **GPU T4 x2** (gratuito)

---

En este notebook exploraremos las capacidades de Llama 3.1, incluyendo:

1. **Carga del modelo** - Configuración básica y cuantizada
2. **Capacidades básicas** - Generación de texto general
3. **Capacidades conversacionales** - Formato de chat
4. **Capacidades multilingües** - Soporte para múltiples idiomas
5. **Generación de código** - Programación asistida por IA
6. **Análisis de rendimiento** - Métricas y optimización
7. **Comparación de configuraciones** - Diferentes parámetros de generación

## 1. Verificación del Entorno y Configuración

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 25.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00:00:0100:01
  Attempting u

In [3]:
# Verificar entorno y configuración
import os
import sys

# Detectar entorno
def detectar_entorno():
    if os.path.exists('/kaggle/input') or 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        return 'Kaggle'
    elif 'COLAB_GPU' in os.environ:
        return 'Google Colab'
    else:
        return 'Local'

entorno = detectar_entorno()
print(f"🔍 Entorno detectado: {entorno}")

from kaggle_secrets import UserSecretsClient

# Accede el secreto "HF_TOKEN"
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# (Opcional) lo setea como variable de entorno para que Transformers lo tome automáticamente
os.environ["HF_TOKEN"] = hf_token
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token  # A veces esta variable también es requerida

print("Token HF seteado correctamente:", hf_token[:10], "...")

# Verificar GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU disponible: {gpu_name}")
    print(f"💾 Memoria GPU: {gpu_memory:.1f} GB")
else:
    print("⚠️ GPU no disponible - usando CPU")

print(f"🐍 Python: {sys.version}")
print(f"🔥 PyTorch: {torch.__version__}")

🔍 Entorno detectado: Kaggle
Token HF seteado correctamente: hf_FLjugcm ...
🚀 GPU disponible: Tesla T4
💾 Memoria GPU: 14.7 GB
🐍 Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
🔥 PyTorch: 2.6.0+cu124


## 2. Configuración Manual del Token (Opcional)

Si necesitas configurar el token manualmente en el notebook, ejecuta la siguiente celda:

In [4]:
# OPCIONAL: Configurar token manualmente si no está en variables de ambiente
# Descomenta y ejecuta solo si es necesario

# import os
# 
# # Reemplaza 'tu_token_aqui' con tu token real de Hugging Face
# # os.environ['HF_TOKEN'] = 'tu_token_aqui'
# 
# # Verificar que se configuró correctamente
# if os.getenv('HF_TOKEN'):
#     print(f"✅ Token configurado: {os.getenv('HF_TOKEN')[:10]}...")
# else:
#     print("⚠️ Token no configurado")

print("💡 Esta celda es opcional. Solo úsala si el token no se detecta automáticamente.")

💡 Esta celda es opcional. Solo úsala si el token no se detecta automáticamente.


## 3. Importaciones y Configuración Inicial

In [5]:
#!/usr/bin/env python3
import torch
import time
import os
import sys
from pathlib import Path
from typing import List, Dict, Optional
import json
import warnings
warnings.filterwarnings('ignore')

# Detectar entorno
def is_kaggle():
    return os.path.exists('/kaggle/input') or 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

def is_colab():
    return 'COLAB_GPU' in os.environ

# Configurar paths según el entorno
if is_kaggle():
    print("🔍 Ejecutando en Kaggle")
    # En Kaggle, el código puede estar en /kaggle/input/
    config_paths = ['/kaggle/input/cursollama311/config', './config', '../config']
elif is_colab():
    print("🔍 Ejecutando en Google Colab")
    config_paths = ['./config', '../config']
else:
    print("🔍 Ejecutando en entorno local")
    config_paths = ['./config', '../config', str(Path.cwd().parent / 'config')]

# Agregar paths de configuración
for path in config_paths:
    if os.path.exists(path):
        sys.path.insert(0, path)
        break

# Instalar dependencias si es necesario (para Kaggle/Colab)
if is_kaggle() or is_colab():
    try:
        import transformers
        print(f"✅ Transformers {transformers.__version__} ya instalado")
    except ImportError:
        print("📦 Instalando transformers...")
        %pip install -q transformers accelerate bitsandbytes

# Importar librerías principales
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

# Importar psutil con fallback
try:
    import psutil
    HAS_PSUTIL = True
except ImportError:
    print("⚠️ psutil no disponible - métricas de memoria limitadas")
    HAS_PSUTIL = False

# Importar matplotlib con fallback
try:
    import matplotlib.pyplot as plt
    HAS_MATPLOTLIB = True
except ImportError:
    print("⚠️ matplotlib no disponible - sin gráficos")
    HAS_MATPLOTLIB = False

# Configuración simplificada usando variables de ambiente del sistema
def setup_huggingface_auth():
    """Configurar autenticación usando variables de ambiente del sistema operativo"""
    # Buscar token en múltiples variables de ambiente
    token = os.getenv('HF_TOKEN') or os.getenv('HUGGINGFACE_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
    
    if token:
        # Configurar todas las variables que transformers puede usar
        os.environ['HUGGING_FACE_HUB_TOKEN'] = token
        os.environ['HF_TOKEN'] = token
        print("🔐 Autenticación HF configurada desde variables de ambiente del sistema")
        return True
    else:
        print("⚠️ No se encontró token HF en variables de ambiente del sistema")
        return False

def get_model_config():
    """Obtener configuración de modelo desde variables de ambiente"""
    return {
        'default_model': os.getenv('DEFAULT_MODEL', 'meta-llama/Meta-Llama-3.1-8B-Instruct'),
        'fallback_model': os.getenv('FALLBACK_MODEL', 'microsoft/DialoGPT-medium'),
        'cache_dir': os.getenv('HF_CACHE_DIR', '/tmp/huggingface_cache' if is_kaggle() else './models_cache'),
        'device': os.getenv('DEVICE', 'auto'),
        'use_quantization': os.getenv('USE_QUANTIZATION', 'true').lower() == 'true'
    }

# Configurar autenticación automáticamente al importar
auth_success = setup_huggingface_auth()
model_config = get_model_config()

print(f"🔧 Configuración desde variables de ambiente:")
print(f"   Autenticación: {'✅ Configurada' if auth_success else '❌ No configurada'}")
print(f"   Modelo: {model_config['default_model']}")
print(f"   Cache: {model_config['cache_dir']}")
print(f"   Cuantización: {model_config['use_quantization']}")

print("📦 Librerías importadas correctamente")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🤗 Transformers: {transformers.__version__}")

🔍 Ejecutando en Kaggle
✅ Transformers 4.52.4 ya instalado


2025-08-01 02:12:34.755450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754014354.969193      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754014355.031275      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔐 Autenticación HF configurada desde variables de ambiente del sistema
🔧 Configuración desde variables de ambiente:
   Autenticación: ✅ Configurada
   Modelo: meta-llama/Meta-Llama-3.1-8B-Instruct
   Cache: /tmp/huggingface_cache
   Cuantización: True
📦 Librerías importadas correctamente
🔥 PyTorch: 2.6.0+cu124
🤗 Transformers: 4.52.4


## 2. Definición de la Clase Llama31Demo

In [9]:
class Llama31Demo:
    """Clase para demostrar las capacidades de Llama 3.1"""
    
    def __init__(self, model_name: str = None):
        # Usar configuración desde variables de ambiente del sistema
        config = get_model_config()
        self.model_name = model_name or config['default_model']
        self.cache_dir = config['cache_dir']
        self.use_quantization = config['use_quantization']
        
        self.tokenizer = None
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        print(f"🦙 Inicializando demostración de Llama 3.1")
        print(f"📱 Dispositivo: {self.device}")
        print(f"🤖 Modelo: {self.model_name}")
        print(f"📁 Cache: {self.cache_dir}")
        print(f"⚡ Cuantización: {self.use_quantization}")
    
    def _obtener_uso_memoria(self) -> float:
        """Obtener uso actual de memoria en MB"""
        process = psutil.Process(os.getpid())
        return process.memory_info().rss / (1024 * 1024)
    
    def _mostrar_info_modelo(self):
        """Mostrar información detallada del modelo"""
        if self.model is None:
            return
        
        # Contar parámetros
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        
        print(f"\n📊 Información del modelo:")
        print(f"   Nombre: {self.model_name}")
        print(f"   Parámetros totales: {total_params:,}")
        print(f"   Parámetros entrenables: {trainable_params:,}")
        print(f"   Dispositivo: {next(self.model.parameters()).device}")
        print(f"   Tipo de datos: {next(self.model.parameters()).dtype}")
        
        # Uso de memoria
        memoria_mb = self._obtener_uso_memoria()
        print(f"   Memoria utilizada: {memoria_mb:.2f} MB")

# Crear instancia de la demo
demo = Llama31Demo()
print("✅ Instancia de Llama31Demo creada")

🦙 Inicializando demostración de Llama 3.1
📱 Dispositivo: cuda
🤖 Modelo: meta-llama/Meta-Llama-3.1-8B-Instruct
📁 Cache: /tmp/huggingface_cache
⚡ Cuantización: True
✅ Instancia de Llama31Demo creada


## 3. Carga del Modelo - Configuración Cuantizada

Primero intentaremos cargar el modelo con cuantización 4-bit para optimizar el uso de memoria.

In [10]:
def cargar_modelo_cuantizado(self):
    """Cargar Llama 3.1 con cuantización para eficiencia"""
    print("\n" + "="*60)
    print("⚡ CARGANDO LLAMA 3.1 - CONFIGURACIÓN CUANTIZADA")
    print("="*60)
    
    try:
        # Configuración de cuantización
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        
        print("🔧 Configuración de cuantización 4-bit activada")
        
        # Cargar tokenizer usando token desde variables de ambiente
        if self.tokenizer is None:
            hf_token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                token=hf_token,
                cache_dir=self.cache_dir
            )
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Cargar modelo cuantizado
        print("🧠 Cargando modelo cuantizado...")
        start_time = time.time()
        
        hf_token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            token=hf_token,
            cache_dir=self.cache_dir
        )
        
        load_time = time.time() - start_time
        
        print(f"✅ Modelo cuantizado cargado en {load_time:.2f} segundos")
        print("💾 Uso de memoria significativamente reducido")
        
        # Mostrar información
        self._mostrar_info_modelo()
        
    except Exception as e:
        print(f"❌ Error cargando modelo cuantizado: {e}")
        if "bitsandbytes" in str(e):
            print("💡 Instalación requerida: pip install bitsandbytes")
        print("🔄 Cargando modelo básico...")
        return False
    return True

# Agregar método a la instancia
Llama31Demo.cargar_modelo_cuantizado = cargar_modelo_cuantizado

# Intentar cargar modelo cuantizado
print("🔄 Intentando cargar modelo cuantizado...")
exito_cuantizado = demo.cargar_modelo_cuantizado()

🔄 Intentando cargar modelo cuantizado...

⚡ CARGANDO LLAMA 3.1 - CONFIGURACIÓN CUANTIZADA
🔧 Configuración de cuantización 4-bit activada


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

🧠 Cargando modelo cuantizado...


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Modelo cuantizado cargado en 178.42 segundos
💾 Uso de memoria significativamente reducido

📊 Información del modelo:
   Nombre: meta-llama/Meta-Llama-3.1-8B-Instruct
   Parámetros totales: 4,540,600,320
   Parámetros entrenables: 1,050,939,392
   Dispositivo: cuda:0
   Tipo de datos: torch.float16
   Memoria utilizada: 3277.27 MB


## 4. Carga del Modelo - Configuración Básica

Si la cuantización falla, cargaremos el modelo en configuración básica.

In [11]:
def cargar_modelo_basico(self):
    """Cargar Llama 3.1 en configuración básica"""
    print("\n" + "="*60)
    print("📥 CARGANDO LLAMA 3.1 - CONFIGURACIÓN BÁSICA")
    print("="*60)
    
    try:
        # Cargar tokenizer usando token desde variables de ambiente
        print("🔤 Cargando tokenizer...")
        hf_token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=hf_token,
            cache_dir=self.cache_dir
        )
        
        # Configurar pad token si no existe
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        print(f"✅ Tokenizer cargado")
        print(f"   Vocabulario: {self.tokenizer.vocab_size:,} tokens")
        print(f"   Tokens especiales: {len(self.tokenizer.special_tokens_map)}")
        
        # Cargar modelo
        print("🧠 Cargando modelo...")
        start_time = time.time()
        
        hf_token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else "cpu",
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            token=hf_token,
            cache_dir=self.cache_dir
        )
        
        load_time = time.time() - start_time
        
        print(f"✅ Modelo cargado en {load_time:.2f} segundos")
        
        # Información del modelo
        self._mostrar_info_modelo()
        
    except Exception as e:
        print(f"❌ Error cargando modelo: {e}")
        print("💡 Intentando con modelo alternativo...")

        # Intentar con modelo alternativo
        try:
            fallback_model = "microsoft/DialoGPT-medium"
            print(f"🔄 Cargando modelo alternativo: {fallback_model}")

            self.tokenizer = AutoTokenizer.from_pretrained(fallback_model)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            self.model = AutoModelForCausalLM.from_pretrained(
                fallback_model,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else "cpu",
                low_cpu_mem_usage=True
            )

            self.model_name = fallback_model
            print(f"✅ Modelo alternativo cargado exitosamente")
            self._mostrar_info_modelo()

        except Exception as fallback_error:
            print(f"❌ Error con modelo alternativo: {fallback_error}")
            print("💡 Sugerencias:")
            print("   - Verifica que tengas acceso al modelo")
            print("   - Asegúrate de tener suficiente memoria RAM")
            print("   - Considera usar cuantización para reducir memoria")

# Agregar método a la instancia
Llama31Demo.cargar_modelo_basico = cargar_modelo_basico

# Si no se cargó el modelo cuantizado, cargar el básico
if not exito_cuantizado or demo.model is None:
    print("🔄 Cargando modelo básico...")
    demo.cargar_modelo_basico()

if demo.model is None:
    print("❌ No se pudo cargar el modelo. Verifica:")
    print("   - Conexión a internet")
    print("   - Acceso al modelo de Hugging Face")
    print("   - Memoria RAM suficiente")
else:
    print("\n🎉 ¡Modelo cargado exitosamente!")


🎉 ¡Modelo cargado exitosamente!


## 5. Funciones de Generación de Texto

In [12]:
def _generar_respuesta(self, prompt: str, max_tokens: int = 100, **kwargs) -> str:
    """Generar respuesta para un prompt dado"""
    try:
        # Configuración por defecto
        config_default = {
            "max_new_tokens": max_tokens,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": self.tokenizer.eos_token_id
        }
        
        # Actualizar con parámetros personalizados
        config_default.update(kwargs)
        
        # Tokenizar entrada
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        
        # Mover a dispositivo correcto
        if self.device == "cuda":
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Generar
        with torch.no_grad():
            outputs = self.model.generate(**inputs, **config_default)
        
        # Decodificar solo los tokens nuevos
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return response.strip()
        
    except Exception as e:
        return f"Error generando respuesta: {e}"

def _generar_respuesta_chat(self, conversacion: List[Dict]) -> str:
    """Generar respuesta usando formato de chat"""
    try:
        # Aplicar chat template
        prompt = self.tokenizer.apply_chat_template(
            conversacion, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        return self._generar_respuesta(prompt, max_tokens=150)
        
    except Exception as e:
        return f"Error en chat: {e}"

def _mostrar_conversacion(self, conversacion: List[Dict]):
    """Mostrar conversación de forma legible"""
    for mensaje in conversacion:
        rol = mensaje["role"]
        contenido = mensaje["content"]
        
        if rol == "system":
            print(f"🔧 Sistema: {contenido}")
        elif rol == "user":
            print(f"👤 Usuario: {contenido}")
        elif rol == "assistant":
            print(f"🤖 Asistente: {contenido}")

# Agregar métodos a la instancia
Llama31Demo._generar_respuesta = _generar_respuesta
Llama31Demo._generar_respuesta_chat = _generar_respuesta_chat
Llama31Demo._mostrar_conversacion = _mostrar_conversacion

print("✅ Funciones de generación agregadas")

✅ Funciones de generación agregadas


## 6. Demostración de Capacidades Básicas

Vamos a probar las capacidades básicas de generación de texto con diferentes tipos de prompts.

In [13]:
def demo_capacidades_basicas(self):
    """Demostrar capacidades básicas de generación"""
    print("\n" + "="*60)
    print("🎯 DEMOSTRACIÓN DE CAPACIDADES BÁSICAS")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado. Ejecuta cargar_modelo_basico() primero.")
        return
    
    # Ejemplos de prompts
    prompts = [
        "Explica qué es la inteligencia artificial en términos simples:",
        "Escribe un código Python para calcular números primos:",
        "¿Cuáles son los beneficios de la energía renovable?",
        "Traduce al inglés: 'La tecnología está cambiando el mundo'",
        "Resuelve: Si tengo 15 manzanas y como 3, ¿cuántas me quedan?"
    ]
    
    for i, prompt in enumerate(prompts, 1):
        print(f"\n🔍 Ejemplo {i}:")
        print(f"Prompt: {prompt}")
        
        # Generar respuesta
        respuesta = self._generar_respuesta(prompt, max_tokens=100)
        print(f"Respuesta: {respuesta}")
        print("-" * 40)

# Agregar método a la instancia
Llama31Demo.demo_capacidades_basicas = demo_capacidades_basicas

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_basicas()
else:
    print("⚠️ Modelo no cargado. Saltando demostración de capacidades básicas.")


🎯 DEMOSTRACIÓN DE CAPACIDADES BÁSICAS

🔍 Ejemplo 1:
Prompt: Explica qué es la inteligencia artificial en términos simples:
Respuesta: una herramienta que puede procesar y analizar grandes cantidades de datos para tomar decisiones informadas. Describe su potencial en diferentes áreas como la medicina, la educación y la seguridad, y analiza sus posibles desafíos y limitaciones. También se discute cómo la IA está mejorando la vida de las personas y cómo puede ser utilizada para resolver problemas complejos.
La inteligencia artificial (IA) es una herramienta que puede procesar y analizar
----------------------------------------

🔍 Ejemplo 2:
Prompt: Escribe un código Python para calcular números primos:
Respuesta: 1. Dado un número entero positivo, encuentre el número primo más cercano a él.
2. Dado un número entero positivo, encuentre el número primo más grande que sea menor que él.
3. Dado un número entero positivo, encuentre el número primo más pequeño que sea mayor que él.

```python


## 7. Demostración de Capacidades Conversacionales

Probemos las capacidades de chat del modelo usando el formato de conversación.

In [14]:
def demo_capacidades_conversacionales(self):
    """Demostrar capacidades conversacionales con formato de chat"""
    print("\n" + "="*60)
    print("💬 DEMOSTRACIÓN DE CAPACIDADES CONVERSACIONALES")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Conversación de ejemplo
    conversacion = [
        {
            "role": "system",
            "content": "Eres un asistente útil y amigable que responde de manera clara y concisa."
        },
        {
            "role": "user", 
            "content": "Hola, ¿puedes explicarme qué es machine learning?"
        }
    ]
    
    print("🗣️ Conversación de ejemplo:")
    self._mostrar_conversacion(conversacion)
    
    # Generar respuesta usando chat template
    respuesta = self._generar_respuesta_chat(conversacion)
    
    conversacion.append({
        "role": "assistant",
        "content": respuesta
    })
    
    print(f"🤖 Asistente: {respuesta}")
    
    # Continuar conversación
    conversacion.append({
        "role": "user",
        "content": "¿Puedes darme un ejemplo práctico?"
    })
    
    print(f"\n👤 Usuario: ¿Puedes darme un ejemplo práctico?")
    
    respuesta2 = self._generar_respuesta_chat(conversacion)
    print(f"🤖 Asistente: {respuesta2}")

# Agregar método a la instancia
Llama31Demo.demo_capacidades_conversacionales = demo_capacidades_conversacionales

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_conversacionales()
else:
    print("⚠️ Modelo no cargado. Saltando demostración conversacional.")


💬 DEMOSTRACIÓN DE CAPACIDADES CONVERSACIONALES
🗣️ Conversación de ejemplo:
🔧 Sistema: Eres un asistente útil y amigable que responde de manera clara y concisa.
👤 Usuario: Hola, ¿puedes explicarme qué es machine learning?
🤖 Asistente: ¡Hola! Claro que sí, me alegra explicarte sobre machine learning.

Machine learning (aprendizaje automático en español) es un subconjunto de la inteligencia artificial (IA) que se enfoca en el desarrollo de algoritmos y modelos computacionales capaces de aprender de datos y mejorar su rendimiento en tareas específicas sin ser explícitamente programados.

En otras palabras, el machine learning permite a los sistemas informáticos tomar decisiones y predecir resultados basándose en patrones y relaciones en los datos, en lugar de seguir una regla o programa fijo. Esto se logra mediante la creación de modelos matemáticos que pueden ser entrenados con datos para

👤 Usuario: ¿Puedes darme un ejemplo práctico?
🤖 Asistente: Claro, aquí te presento un ejemplo práct

## 8. Demostración de Capacidades Multilingües

Exploremos las capacidades del modelo en diferentes idiomas.

In [15]:
def demo_capacidades_multilingues(self):
    """Demostrar capacidades multilingües"""
    print("\n" + "="*60)
    print("🌍 DEMOSTRACIÓN DE CAPACIDADES MULTILINGÜES")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Prompts en diferentes idiomas
    prompts_multilingues = {
        "Español": "Describe las ventajas de la energía solar:",
        "English": "Explain the benefits of renewable energy:",
        "Français": "Expliquez les avantages de l'énergie solaire:",
        "Deutsch": "Erklären Sie die Vorteile der Solarenergie:",
        "Italiano": "Spiega i vantaggi dell'energia solare:",
        "Português": "Explique as vantagens da energia solar:"
    }
    
    for idioma, prompt in prompts_multilingues.items():
        print(f"\n🗣️ {idioma}:")
        print(f"Prompt: {prompt}")
        
        respuesta = self._generar_respuesta(prompt, max_tokens=80)
        print(f"Respuesta: {respuesta}")
        print("-" * 40)

# Agregar método a la instancia
Llama31Demo.demo_capacidades_multilingues = demo_capacidades_multilingues

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_multilingues()
else:
    print("⚠️ Modelo no cargado. Saltando demostración multilingüe.")


🌍 DEMOSTRACIÓN DE CAPACIDADES MULTILINGÜES

🗣️ Español:
Prompt: Describe las ventajas de la energía solar:
Respuesta: Las ventajas de la energía solar son varias y se pueden resumir en las siguientes:
1. **Renovable y sostenible**: La energía solar es una fuente de energía renovable y sostenible, ya que no se agota con el uso y no emite gases de efecto invernadero.
2. **Costo reducido**: El costo de
----------------------------------------

🗣️ English:
Prompt: Explain the benefits of renewable energy:
Respuesta: Renewable energy is a vital component in the transition to a low-carbon economy. The benefits of renewable energy include:
Reduced greenhouse gas emissions: Renewable energy sources emit little to no greenhouse gases, contributing to the reduction of climate change.
Energy independence: Renewable energy can reduce reliance on imported fossil fuels, enhancing energy security and reducing trade deficits.
Job creation and economic growth: The renewable energy industry is creating

## 9. Demostración de Capacidades de Código

Probemos las capacidades del modelo para generar código en diferentes lenguajes de programación.

In [16]:
def demo_capacidades_codigo(self):
    """Demostrar capacidades de generación de código"""
    print("\n" + "="*60)
    print("💻 DEMOSTRACIÓN DE CAPACIDADES DE CÓDIGO")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Prompts de programación
    prompts_codigo = [
        "Escribe una función Python para calcular el factorial de un número:",
        "Crea una clase JavaScript para manejar una lista de tareas:",
        "Escribe una consulta SQL para obtener los 10 productos más vendidos:",
        "Implementa un algoritmo de búsqueda binaria en Python:"
    ]
    
    for i, prompt in enumerate(prompts_codigo, 1):
        print(f"\n💻 Ejemplo de código {i}:")
        print(f"Prompt: {prompt}")
        
        respuesta = self._generar_respuesta(prompt, max_tokens=200)
        print(f"Código generado:\n{respuesta}")
        print("-" * 50)

# Agregar método a la instancia
Llama31Demo.demo_capacidades_codigo = demo_capacidades_codigo

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_codigo()
else:
    print("⚠️ Modelo no cargado. Saltando demostración de código.")


💻 DEMOSTRACIÓN DE CAPACIDADES DE CÓDIGO

💻 Ejemplo de código 1:
Prompt: Escribe una función Python para calcular el factorial de un número:
Código generado:
```python
def factorial(n):
    """
    Calcula el factorial de un número entero n.
    
    Args:
    n (int): El número entero para el que se calculará el factorial.
    
    Returns:
    int: El factorial de n.
    
    Raises:
    ValueError: Si n es negativo.
    TypeError: Si n no es un número entero.
    """
    if not isinstance(n, int):
        raise TypeError("n debe ser un número entero.")
    if n < 0:
        raise ValueError("n debe ser no negativo.")
    elif n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)
```
Aquí hay un ejemplo de cómo se utiliza la función:
```python
print(factorial(5))  # Salida: 120
print(factorial(0))  # Salida: 1
print(factorial(1
--------------------------------------------------

💻 Ejemplo de código 2:
Prompt: Crea una clase JavaScript para manejar una lista d

## 10. Análisis de Rendimiento

Analicemos el rendimiento del modelo midiendo tiempo de respuesta y uso de memoria.

In [17]:
def demo_analisis_rendimiento(self):
    """Analizar rendimiento del modelo"""
    print("\n" + "="*60)
    print("📊 ANÁLISIS DE RENDIMIENTO")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Test de rendimiento
    prompt_test = "Explica el concepto de inteligencia artificial y sus aplicaciones en la industria moderna:"
    
    print(f"🧪 Prompt de prueba: {prompt_test}")
    
    # Medir tiempo y memoria
    memoria_inicial = self._obtener_uso_memoria()
    
    tiempos = []
    longitudes = []
    
    for i in range(3):
        print(f"\n🔄 Ejecución {i+1}/3:")
        
        start_time = time.time()
        respuesta = self._generar_respuesta(prompt_test, max_tokens=150)
        end_time = time.time()
        
        tiempo_generacion = end_time - start_time
        longitud_respuesta = len(respuesta.split())
        
        tiempos.append(tiempo_generacion)
        longitudes.append(longitud_respuesta)
        
        tokens_por_segundo = longitud_respuesta / tiempo_generacion if tiempo_generacion > 0 else 0
        
        print(f"   Tiempo: {tiempo_generacion:.2f}s")
        print(f"   Palabras generadas: {longitud_respuesta}")
        print(f"   Velocidad: {tokens_por_segundo:.2f} palabras/s")
    
    memoria_final = self._obtener_uso_memoria()
    
    # Estadísticas finales
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"   Tiempo promedio: {sum(tiempos)/len(tiempos):.2f}s")
    print(f"   Velocidad promedio: {sum(longitudes)/sum(tiempos):.2f} palabras/s")
    print(f"   Uso de memoria: {memoria_final - memoria_inicial:.2f} MB")
    print(f"   Memoria total usada: {memoria_final:.2f} MB")

# Agregar método a la instancia
Llama31Demo.demo_analisis_rendimiento = demo_analisis_rendimiento

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_analisis_rendimiento()
else:
    print("⚠️ Modelo no cargado. Saltando análisis de rendimiento.")


📊 ANÁLISIS DE RENDIMIENTO
🧪 Prompt de prueba: Explica el concepto de inteligencia artificial y sus aplicaciones en la industria moderna:

🔄 Ejecución 1/3:
   Tiempo: 15.31s
   Palabras generadas: 95
   Velocidad: 6.20 palabras/s

🔄 Ejecución 2/3:
   Tiempo: 15.13s
   Palabras generadas: 96
   Velocidad: 6.35 palabras/s

🔄 Ejecución 3/3:
   Tiempo: 15.08s
   Palabras generadas: 92
   Velocidad: 6.10 palabras/s

📈 ESTADÍSTICAS FINALES:
   Tiempo promedio: 15.17s
   Velocidad promedio: 6.22 palabras/s
   Uso de memoria: 0.00 MB
   Memoria total usada: 3552.39 MB


## 11. Comparación de Configuraciones de Generación

Comparemos diferentes configuraciones de generación para ver cómo afectan la creatividad y coherencia del modelo.

In [18]:
def demo_comparacion_configuraciones(self):
    """Comparar diferentes configuraciones de generación"""
    print("\n" + "="*60)
    print("⚙️ COMPARACIÓN DE CONFIGURACIONES")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    prompt = "Escribe un párrafo sobre el futuro de la tecnología:"
    
    configuraciones = {
        "Conservadora": {"temperature": 0.3, "top_p": 0.8, "do_sample": True},
        "Balanceada": {"temperature": 0.7, "top_p": 0.9, "do_sample": True},
        "Creativa": {"temperature": 1.0, "top_p": 0.95, "do_sample": True},
        "Determinística": {"temperature": 0.0, "do_sample": False}
    }
    
    print(f"🎯 Prompt: {prompt}")
    
    for nombre, config in configuraciones.items():
        print(f"\n🔧 Configuración {nombre}:")
        print(f"   Parámetros: {config}")
        
        respuesta = self._generar_respuesta(prompt, max_tokens=100, **config)
        print(f"   Respuesta: {respuesta}")
        print("-" * 50)

# Agregar método a la instancia
Llama31Demo.demo_comparacion_configuraciones = demo_comparacion_configuraciones

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_comparacion_configuraciones()
else:
    print("⚠️ Modelo no cargado. Saltando comparación de configuraciones.")


⚙️ COMPARACIÓN DE CONFIGURACIONES
🎯 Prompt: Escribe un párrafo sobre el futuro de la tecnología:

🔧 Configuración Conservadora:
   Parámetros: {'temperature': 0.3, 'top_p': 0.8, 'do_sample': True}
   Respuesta: ¿qué cambios se esperan en el futuro?
La tecnología está en constante evolución y se espera que siga avanzando a un ritmo acelerado en el futuro. Algunos de los cambios que se esperan incluyen la adopción generalizada de la inteligencia artificial (IA) en diversas áreas de la vida, como la medicina, la educación y la industria. Además, se espera que la realidad virtual (RV) y la realidad aumentada
--------------------------------------------------

🔧 Configuración Balanceada:
   Parámetros: {'temperature': 0.7, 'top_p': 0.9, 'do_sample': True}
   Respuesta: ¿qué cambios se esperan y cómo podrían afectar a la sociedad?
La tecnología seguirá evolucionando a un ritmo acelerado en los próximos años, impulsada por la innovación y la investigación en áreas como la inteligencia artifi

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   Respuesta: En el futuro cercano, es probable que avancemos en la tecnología de la información y comunicación (TIC) a velocidades cada vez más rápidas, llevándonos a una era de realidad virtual y augmented, donde la vida se voltea y la vida cotidiana se integra con los datos y aplicaciones. Las redes de 5G permitirán que los datos se transmitan a velocidades más altas, lo que facilitará la interacción
--------------------------------------------------

🔧 Configuración Determinística:
   Parámetros: {'temperature': 0.0, 'do_sample': False}
   Respuesta: ¿qué cambios podemos esperar en los próximos años?
La tecnología está en constante evolución y es probable que en los próximos años experimentemos cambios significativos en diversas áreas. Una de las tendencias más destacadas es la adopción de la inteligencia artificial (IA) en la vida diaria, lo que podría llevar a la creación de sistemas de asistencia personalizados y la automatización de tareas rutinarias. Además, la realidad aument

## 12. Resumen y Conclusiones

¡Felicidades! Has completado la demostración de Llama 3.1. En este notebook hemos explorado:

### ✅ Lo que hemos cubierto:

1. **Carga del modelo** - Tanto en configuración básica como cuantizada
2. **Capacidades básicas** - Generación de texto general
3. **Capacidades conversacionales** - Formato de chat estructurado
4. **Capacidades multilingües** - Soporte para múltiples idiomas
5. **Generación de código** - Programación asistida por IA
6. **Análisis de rendimiento** - Métricas de velocidad y memoria
7. **Configuraciones de generación** - Parámetros de creatividad y coherencia

### 🎯 Puntos clave aprendidos:

- **Llama 3.1** es un modelo muy capaz para múltiples tareas
- La **cuantización** puede reducir significativamente el uso de memoria
- Los **parámetros de generación** afectan la creatividad vs coherencia
- El modelo maneja bien **múltiples idiomas** y **generación de código**
- El **formato de chat** permite conversaciones más naturales
- **Variables de ambiente** proporcionan configuración segura y portable
- La configuración automática funciona en **Kaggle, Colab y entorno local**

### 🚀 Próximos pasos:

- Experimenta con diferentes prompts y configuraciones
- Prueba el modelo en tus propios casos de uso
- Explora técnicas de fine-tuning para tareas específicas
- Considera la implementación en aplicaciones reales

### 📚 Recursos adicionales:

- [Documentación de Transformers](https://huggingface.co/docs/transformers)
- [Llama 3.1 Model Card](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct)
- [Guías de optimización](https://huggingface.co/docs/transformers/perf_infer_gpu_one)

---

**¡Gracias por completar el Módulo 2 del Curso de Llama 3.1!** 🎉

In [19]:
# Limpieza final
print("\n🧹 Limpieza de memoria...")
if 'demo' in locals() and demo.model is not None:
    del demo.model
    del demo.tokenizer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    print("✅ Memoria liberada")

print("\n🎓 ¡Módulo 2 completado exitosamente!")
print("📝 Notebook guardado como: Modulo2_Llama31_Demo.ipynb")


🧹 Limpieza de memoria...
✅ Memoria liberada

🎓 ¡Módulo 2 completado exitosamente!
📝 Notebook guardado como: Modulo2_Llama31_Demo.ipynb
